In [1]:
import sys
print(sys.executable)

/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/bin/python3.10


In [2]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import pandas as pd
import numpy as np

# === 1. Chargement des données ===
df1 = pd.read_csv("dataset/df_agrege_.csv")



/tmp/ipykernel_890/676978158.py:12: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("dataset/df_agrege_.csv")


In [3]:
df = df1
# Creation du pipeline de ML


types_biens = ['Appartement', 'Maison']
departements = ['01', '26', '31', '33', '35', '38', '44', '45', '63', '69']
résultats = []

df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
df['code_postal'] = df['code_postal'].astype(str)
df['code_departement'] = df['code_postal'].str[:2]

for type_local in types_biens:
    for dep in departements:
        print(f"\n🔍 {type_local} - {dep}")
        
        df_sub = df[
            (df['date_mutation'].dt.year == 2024) &
            (df['code_departement'] == dep) &
            (df['type_local'] == type_local) &
            (df['valeur_fonciere'] > 0) &
            (df['valeur_fonciere'] < 1_500_000) &
            (df['surface_reelle_bati'] > 10) &
            (df['surface_reelle_bati'] < 350)
        ].copy()

        if df_sub.shape[0] < 300:
            print("⚠️ Trop peu de données, on saute ce binôme.")
            continue

        df_sub = df_sub[df_sub['nombre_pieces_principales'] > 0]
        df_sub['prix_m2'] = df_sub['valeur_fonciere'] / df_sub['surface_reelle_bati']
        df_sub['surface_par_piece'] = df_sub['surface_reelle_bati'] / df_sub['nombre_pieces_principales']

        # 🧼 Suppression des outliers extrêmes (1er et 99e percentile)
        q1, q99 = df_sub['prix_m2'].quantile([0.01, 0.99])
        df_sub = df_sub[df_sub['prix_m2'].between(q1, q99)]

        # 🏙️ Typologie géographique selon le prix/m²
        def tag_zone(val):
            if val < 2000:
                return 'zone_rurale'
            elif val <= 3000:
                return 'zone_intermediaire'
            else:
                return 'zone_urbaine'

        df_sub['zone_typologique'] = df_sub['prix_m2'].apply(tag_zone)

        # 📊 Stat locale
        prix_medians = df_sub.groupby('code_postal')['prix_m2'].median().rename('prix_m2_median_code_postal')
        df_sub = df_sub.merge(prix_medians, on='code_postal', how='left')
        df_sub['prix_m2_median_code_postal'] = df_sub['prix_m2_median_code_postal'].fillna(df_sub['prix_m2'].median())

        df_sub = df_sub.dropna(subset=[
            'surface_reelle_bati', 'nombre_pieces_principales',
            'surface_par_piece', 'code_postal',
            'prix_m2_median_code_postal', 'prix_m2',
            'latitude', 'longitude', 'zone_typologique'
        ])

        # 📋 Features
        features = ['surface_reelle_bati', 'nombre_pieces_principales', 'surface_par_piece',
                    'code_postal', 'prix_m2_median_code_postal',
                    'latitude', 'longitude', 'zone_typologique']
        target = 'prix_m2'
        X = df_sub[features]
        y = df_sub[target]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # ⚙️ Pipeline enrichi
        preproc = ColumnTransformer([
            ('encode_postal', TargetEncoder(), ['code_postal']),
            ('encode_zone', OneHotEncoder(drop='first'), ['zone_typologique']),
            ('scale_num', StandardScaler(), [
                'surface_reelle_bati', 'nombre_pieces_principales', 'surface_par_piece',
                'prix_m2_median_code_postal', 'latitude', 'longitude'
            ])
        ])

        pipeline = Pipeline(steps=[
            ('prep', preproc),
            ('reg', XGBRegressor(n_estimators=150, learning_rate=0.1,
                                 max_depth=5, random_state=42))
        ])

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        # 🔁 Hybridation avec stat locale
        medianes_test = X_test['prix_m2_median_code_postal'].values
        y_hybrid = 0.7 * y_pred + 0.3 * medianes_test

        r2 = r2_score(y_test, y_hybrid)
        mae = mean_absolute_error(y_test, y_hybrid)
        mape = mean_absolute_percentage_error(y_test, y_hybrid) * 100

        print(f"✅ R² = {r2:.3f} | MAE = {mae:,.0f} €/m² | MAPE = {mape:.2f}%")

        résultats.append({
            'type_local': type_local,
            'departement': dep,
            'r2': round(r2, 3),
            'mae': int(mae),
            'mape': round(mape, 2)
        })



🔍 Appartement - 01
⚠️ Trop peu de données, on saute ce binôme.

🔍 Appartement - 26


/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


✅ R² = 0.715 | MAE = 272 €/m² | MAPE = 18.72%

🔍 Appartement - 31


/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


✅ R² = 0.802 | MAE = 367 €/m² | MAPE = 14.58%

🔍 Appartement - 33


...

✅ R² = 0.739 | MAE = 299 €/m² | MAPE = 21.83%

🔍 Maison - 63


/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


✅ R² = 0.824 | MAE = 387 €/m² | MAPE = 44.17%

🔍 Maison - 69


/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


✅ R² = 0.690 | MAE = 602 €/m² | MAPE = 20.58%


In [4]:
# Affichage des resultats du pipeline de ML. Objectif: avoir un R2 elevé et MAE et MAPE faible
pd.DataFrame(résultats).sort_values(by='mape')


,type_local,departement,r2,mae,mape
3,Appartement,35,0.742,432,12.98
5,Appartement,44,0.705,480,14.08
2,Appartement,33,0.730,527,14.40
1,Appartement,31,0.802,367,14.58
6,Appartement,45,0.807,262,14.79
4,Appartement,38,0.795,338,15.35
8,Appartement,69,0.696,528,15.43
14,Maison,44,0.753,439,16.41
13,Maison,38,0.756,370,16.75
0,Appartement,26,0.715,272,18.72


In [ ]:
#Le modele fonctionne mieux pour les appartements car les appartements sont dans des zones + denses donc davantage de données pour entrainer le modele

In [5]:
import pandas as pd
import joblib

# Création d'une fonction pour estimer la valeur d'un bien

def estimer_bien(bien: dict, modele_dict: dict, stats_locales: pd.DataFrame,
                 alpha: float = 0.7) -> dict:
    """
    Estime le prix d’un bien en €/m² et sa valeur totale, en combinant un modèle ML et une stat locale.

    Params :
        bien : dict avec les clés suivantes :
            - type_local : 'Appartement' ou 'Maison'
            - code_postal : str ou int
            - surface_reelle_bati : float
            - nombre_pieces_principales : int
            - latitude : float
            - longitude : float
        modele_dict : dict {(type_local, departement): pipeline}
        stats_locales : DataFrame avec colonnes ['code_postal', 'prix_m2_median_code_postal']
        alpha : float (poids du modèle ML dans l’estimation finale)

    Return :
        dict avec prix estimé, fourchette, commentaire
    """
    # 🔎 Extraction
    type_local = bien['type_local']
    code_postal = str(bien['code_postal'])
    dep = code_postal[:2]

    # 📦 Récupération du modèle correspondant
    key = (type_local, dep)
    if key not in modele_dict:
        return {"erreur": f"❌ Modèle non disponible pour {type_local} - {dep}"}

    model = modele_dict[key]

    # 💡 Création des features enrichies
    surface = bien['surface_reelle_bati']
    pieces = bien['nombre_pieces_principales']
    surface_par_piece = surface / pieces if pieces > 0 else 0

    prix_median_local = stats_locales.loc[
        stats_locales['code_postal'] == code_postal, 'prix_m2_median_code_postal'
    ]

    prix_m2_median = prix_median_local.values[0] if not prix_median_local.empty else None

    # 🧭 Zone typologique
    if prix_m2_median is not None:
        if prix_m2_median < 2000:
            zone = 'zone_rurale'
        elif prix_m2_median <= 3000:
            zone = 'zone_intermediaire'
        else:
            zone = 'zone_urbaine'
    else:
        zone = 'zone_intermediaire'

    # 🧪 Donnée formatée pour prédiction
    X_input = pd.DataFrame([{
        'surface_reelle_bati': surface,
        'nombre_pieces_principales': pieces,
        'surface_par_piece': surface_par_piece,
        'code_postal': code_postal,
        'prix_m2_median_code_postal': prix_m2_median or 2500,
        'latitude': bien['latitude'],
        'longitude': bien['longitude'],
        'zone_typologique': zone
    }])

    # 🤖 Prédiction ML
    prix_m2_ml = model.predict(X_input)[0]

    # ⚖️ Estimation hybride
    if prix_m2_median:
        prix_m2_final = alpha * prix_m2_ml + (1 - alpha) * prix_m2_median
    else:
        prix_m2_final = prix_m2_ml

    valeur_fonciere_estimee = int(prix_m2_final * surface)
    prix_min = int(valeur_fonciere_estimee * 0.85)
    prix_max = int(valeur_fonciere_estimee * 1.15)

    # 💬 Commentaire
    if prix_m2_median:
        ratio = prix_m2_final / prix_m2_median
        if ratio < 0.9:
            commentaire = "🔽 Estimation sous le marché local"
        elif ratio > 1.1:
            commentaire = "🔼 Estimation au-dessus du marché local"
        else:
            commentaire = "✅ Estimation cohérente avec les prix locaux"
    else:
        commentaire = "ℹ️ Pas de référence de prix local disponible"

    return {
        'prix_m2_estime': round(prix_m2_final, 2),
        'valeur_fonciere_estimee': valeur_fonciere_estimee,
        'fourchette': (prix_min, prix_max),
        'zone': zone,
        'commentaire': commentaire
    }


In [6]:
import joblib

# Génération du dictionnaire de modèle utilisé par la fonction estimer_bien()

modele_dict = {}
stats_locaux_list = []

for type_local in types_biens:
    for dep in departements:
        df_sub = df[
            (df['date_mutation'].dt.year == 2024) &
            (df['code_departement'] == dep) &
            (df['type_local'] == type_local) &
            (df['valeur_fonciere'] > 0) &
            (df['valeur_fonciere'] < 1_500_000) &
            (df['surface_reelle_bati'] > 10) &
            (df['surface_reelle_bati'] < 350)
        ].copy()

        if df_sub.shape[0] < 300:
            continue

        df_sub = df_sub[df_sub['nombre_pieces_principales'] > 0]
        df_sub['prix_m2'] = df_sub['valeur_fonciere'] / df_sub['surface_reelle_bati']
        df_sub['surface_par_piece'] = df_sub['surface_reelle_bati'] / df_sub['nombre_pieces_principales']
        
        q1, q99 = df_sub['prix_m2'].quantile([0.01, 0.99])
        df_sub = df_sub[df_sub['prix_m2'].between(q1, q99)]

        prix_medians = df_sub.groupby('code_postal')['prix_m2'].median().rename('prix_m2_median_code_postal')
        df_sub = df_sub.merge(prix_medians, on='code_postal', how='left')
        df_sub['prix_m2_median_code_postal'] = df_sub['prix_m2_median_code_postal'].fillna(df_sub['prix_m2'].median())

        def tag_zone(val):
            if val < 2000:
                return 'zone_rurale'
            elif val <= 3000:
                return 'zone_intermediaire'
            else:
                return 'zone_urbaine'

        df_sub['zone_typologique'] = df_sub['prix_m2_median_code_postal'].apply(tag_zone)

        df_sub = df_sub.dropna(subset=[
            'surface_reelle_bati', 'nombre_pieces_principales', 'surface_par_piece',
            'code_postal', 'prix_m2_median_code_postal', 'latitude', 'longitude'
        ])

        features = ['surface_reelle_bati', 'nombre_pieces_principales', 'surface_par_piece',
                    'code_postal', 'prix_m2_median_code_postal', 'latitude', 'longitude', 'zone_typologique']
        target = 'prix_m2'
        X = df_sub[features]
        y = df_sub[target]

        X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

        preproc = ColumnTransformer([
            ('encode_cp', TargetEncoder(), ['code_postal']),
            ('encode_zone', OneHotEncoder(drop='first'), ['zone_typologique']),
            ('scale_num', StandardScaler(), [
                'surface_reelle_bati', 'nombre_pieces_principales',
                'surface_par_piece', 'prix_m2_median_code_postal', 'latitude', 'longitude'
            ])
        ])

        pipeline = Pipeline([
            ('prep', preproc),
            ('reg', XGBRegressor(n_estimators=150, learning_rate=0.1, max_depth=5, random_state=42))
        ])

        pipeline.fit(X_train, y_train)
        modele_dict[(type_local, dep)] = pipeline

        # Export stats locales aussi
        stats_zone = df_sub[['code_postal', 'prix_m2_median_code_postal']].drop_duplicates()
        stats_locaux_list.append(stats_zone)


/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/mnt/c/Users/DELL/Desktop/P3_Data_Hub/mage_env/lib/python3.10/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf wi

In [7]:
# Assemblage des statistiques locales requises pour utiliser la fonction estimer_bien()
stats_locales = pd.concat(stats_locaux_list).drop_duplicates('code_postal')


In [13]:
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
# ajout d'une fonction pour retourner la latitude et la longitude depuis une adresse rentrée par l'utilisateur
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

geolocator = Nominatim(user_agent="estimation-immo-gps")

def geocoder_adresse(adresse, tentative=1, max_tentative=3):
    try:
        location = geolocator.geocode(adresse, timeout=10)
        if location:
            return {'latitude': location.latitude, 'longitude': location.longitude}
        else:
            return {'latitude': None, 'longitude': None}
    except GeocoderTimedOut:
        if tentative <= max_tentative:
            time.sleep(1)
            return geocoder_adresse(adresse, tentative+1)
        return {'latitude': None, 'longitude': None}


In [9]:
# test fonction geocoder_adresse avec une adresse:
adresse = "13 rue victor hugo, 35000 Rennes"
coord = geocoder_adresse(adresse)

print(f"🌍 Coordonnées : {coord}")


🌍 Coordonnées : {'latitude': 48.1125073, 'longitude': -1.6759495}


In [10]:
def estimer_depuis_adresse(adresse_str, type_local, surface, nb_pieces, code_postal,
                           modele_dict, stats_locales, alpha=0.7):
    """
    Estime un bien immobilier à partir d'une adresse complète
    """
    coords = geocoder_adresse(adresse_str)

    if coords['latitude'] is None or coords['longitude'] is None:
        return {"erreur": f"❌ Adresse introuvable via Nominatim : {adresse_str}"}

    bien = {
        'type_local': type_local,
        'code_postal': str(code_postal),
        'surface_reelle_bati': surface,
        'nombre_pieces_principales': nb_pieces,
        'latitude': coords['latitude'],
        'longitude': coords['longitude']
    }

    resultat = estimer_bien(bien, modele_dict, stats_locales, alpha)

    return {
        'adresse': adresse_str,
        'prix_m2_estime': resultat['prix_m2_estime'],
        'valeur_fonciere_estimee': resultat['valeur_fonciere_estimee'],
        'fourchette': resultat['fourchette'],
        'zone': resultat['zone'],
        'commentaire': resultat['commentaire']
    }


In [11]:
résultat = estimer_depuis_adresse(
    adresse_str="30 rue alsace-lorraine, 45000 orleans",
    type_local='Appartement',
    surface=65,
    nb_pieces=3,
    code_postal='45000',
    modele_dict=modele_dict,
    stats_locales=stats_locales
)

print("🏘️ Estimation automatique basée sur l'adresse :")
print(f"📍 {résultat['adresse']}")
print(f"📏 Prix/m² : {résultat['prix_m2_estime']} €")
print(f"💶 Estimation totale : {résultat['valeur_fonciere_estimee']:,} €")
print(f"📎 Fourchette : {résultat['fourchette'][0]:,} € – {résultat['fourchette'][1]:,} €")


🏘️ Estimation automatique basée sur l'adresse :
📍 30 rue alsace-lorraine, 45000 orleans
📏 Prix/m² : 2807.0 €
💶 Estimation totale : 182,455 €
📎 Fourchette : 155,086 € – 209,823 €


In [12]:
import joblib
import os
import pandas as pd

# Créer le dossier s'il n'existe pas
os.makedirs("dataset", exist_ok=True)

# Sauvegarder avec le protocole 4 pour une meilleure compatibilité
joblib.dump(modele_dict, "dataset/modele_dict.pkl", protocol=4)
stats_locales.to_pickle("dataset/stats_locales.pkl", protocol=4)

# Vérifier que les fichiers ont été créés
print("Fichiers sauvegardés :")
print(f"- {os.path.abspath('dataset/modele_dict.pkl')}")
print(f"- {os.path.abspath('dataset/stats_locales.pkl')}")

Fichiers sauvegardés :
- /mnt/c/Users/DELL/Desktop/P3_Data_Hub/dataset/modele_dict.pkl
- /mnt/c/Users/DELL/Desktop/P3_Data_Hub/dataset/stats_locales.pkl


In [18]:
import joblib
import os


# 💾 Sauvegarde des objets en .pkl
joblib.dump(modele_dict, "dataset/modele_dict.pkl")
stats_locales.to_pickle("dataset/stats_locales.pkl")
